In [2]:
# !python font2img.py --ttf_path ttf_folder --chara total_chn.txt --save_path save_folder --img_size 80 --chara_size 60

usage: ipykernel_launcher.py [-h] [--ttf_path TTF_PATH] [--chara CHARA]
                             [--save_path SAVE_PATH] [--img_size IMG_SIZE]
                             [--chara_size CHARA_SIZE]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/hensonchen/Library/Jupyter/runtime/kernel-c5d4fde8-5fcc-45f9-acd9-14890cc51eca.json


SystemExit: 2

/Users/hensonchen/Documents/pytorch-test/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
class args:
    ttf_path = "ttf_folder"
    # calli_path = "calli_folder"
    chara = "total_chn_traditional.txt"
    save_path = "save_folder"
    img_size = 80
    chara_size = 60
    num_calli = 300
    urls_cid = [38, 27, 40, 37, 29, 27, 34, 39, 25, 43]
    url_format = "http://163.20.160.14/~word/modules/myalbum/viewcat.php?num=" + str(num_calli)

In [5]:
from PIL import Image,ImageDraw,ImageFont
import matplotlib.pyplot as plt
import os
import numpy as np
import pathlib
# import argparse
from fontTools.ttLib import TTFont
import torchvision.transforms as T

# parser = argparse.ArgumentParser(description='Obtaining characters from .ttf')
# parser.add_argument('--ttf_path', type=str, default='../ttf_folder',help='ttf directory')
# parser.add_argument('--chara', type=str, default='../chara.txt',help='characters')
# parser.add_argument('--save_path', type=str, default='../save_folder',help='images directory')
# parser.add_argument('--img_size', type=int, help='The size of generated images')
# parser.add_argument('--chara_size', type=int, help='The size of generated characters')
# args = parser.parse_args()

# def of preprocessor to make the image 140 * 140
preprocess_raw = T.Compose([
    T.Grayscale(),
    T.Pad(100, 255),
    T.CenterCrop(140),
    T.Resize((args.img_size, args.img_size)),
    #   T.ToTensor(),
    # T.Normalize(mean=0, std=1) #???????
])

file_object = open(args.chara, encoding='utf-8')   
try:
    characters = file_object.read()
finally:
    file_object.close()


def draw_single_char(ch, font, canvas_size, x_offset, y_offset):
    img = Image.new("RGB", (canvas_size, canvas_size), (255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.text((x_offset, y_offset), ch, (0, 0, 0), font=font)
    img = preprocess_raw(img)
    return img

def draw_example(ch, src_font, canvas_size, x_offset, y_offset):
    src_img = draw_single_char(ch, src_font, canvas_size, x_offset, y_offset)
    example_img = Image.new("RGB", (canvas_size, canvas_size), (255, 255, 255))
    example_img.paste(src_img, (0, 0))
    return example_img

data_dir = args.ttf_path
data_root = pathlib.Path(data_dir)
print(data_root)

all_image_paths = list(data_root.glob('*.*'))  # *.ttf TTF
all_image_paths = [str(path) for path in all_image_paths]
total_num = len(all_image_paths)
print(total_num)

seq = list()

if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)

def get_char_list_from_ttf(font_file):
    f_obj = TTFont(font_file)
    m_dict = f_obj.getBestCmap()

    unicode_list = []
    for key, uni in m_dict.items():
        unicode_list.append(key)

    char_list = [chr(ch_unicode) for ch_unicode in unicode_list]
    return char_list

for idx, (label, item) in enumerate(zip(range(len(all_image_paths)),all_image_paths)):
    print("{} / {} ".format(idx, total_num), item)
    src_font = ImageFont.truetype(item, size=args.chara_size)
    font_name = item.split('/')[-1].split('.')[0]
    chars = get_char_list_from_ttf(item)  #
    img_cnt = 0
    for (chara, cnt) in zip(characters, range(len(characters))):
        img = draw_example(chara, src_font, args.img_size, (args.img_size-args.chara_size)/2, (args.img_size-args.chara_size)/2)
        path_full = os.path.join(args.save_path, 'id_%d'%(label))
        if not os.path.exists(path_full):
            os.mkdir(path_full)
        if np.sum(np.array(img) / 255.) < 18000:
            img_cnt += 1
            img.save(os.path.join(path_full, "%05d.png" % (cnt)))



ttf_folder
1
0 / 1  ttf_folder/NotoSansTC-Medium.otf


In [6]:
import requests
from bs4 import BeautifulSoup

In [11]:
# create a dictionary of word list from `characters`, with (key, value) = (character, position_count)
chara_dict = {}
for (chara, cnt) in zip(characters, range(len(characters))):
    chara_dict[chara] = cnt

def download_data_with_no_check(path_full, soup):
    # if not os.path.isdir(args.calli_path):
    #     os.mkdir(Config.RAW_DIR)
    #     os.mkdir(Config.TRAIN_DIR)
    #     os.mkdir(Config.VAL_DIR)
    #     os.mkdir(Config.EVAL_DIR)

  # get all raw image tags and save them to directory "zhiyong"
    def has_title(tag):
        return tag.name == "img" and tag.has_attr('title')
    raw_img_tags = soup.find_all(has_title)
    print(raw_img_tags)
    for raw_img_tag in raw_img_tags:
        if "http" in raw_img_tag.get('src'):
            lnk = raw_img_tag.get('src')
            title = raw_img_tag.get('title')
            if len(title) != 1 or title not in chara_dict:
              continue
            pos = chara_dict[title]
            img = Image.open(requests.get(lnk, stream = True).raw)
            img = preprocess_raw(img)
            img.save(os.path.join(path_full, "%05d.png" % (pos)))
            # with open(os.path.join(path_full, "%05d.gif" % (pos)), "wb") as f:
            #     f.write(requests.get(lnk).content)

    # loop through the titles and remove files with name length > 5
    for filename in os.listdir(path_full):
        # here it's 5 because we include ".gif"
        if len(filename) > 9 and os.path.isfile(path_full+"/"+filename) :
            f = os.path.join(path_full, filename)
            os.remove(f)

def download_data(path_full, soup):
    if os.path.exists(path_full) and not os.path.isfile(path_full):
        # Checking if the directory is empty or not
        lstdir = os.listdir(path_full)
        if not lstdir or (len(lstdir) < 3 and lstdir[0][0] == '.'): # empty dir
            print("data dir emtpy, downloading data")
            download_data_with_no_check(path_full, soup)
        else:
            print("data dir not empty, skipping download")
            pass
    else:
        print("data dir does not exist, creating dir and downloading data")
        os.mkdir(path_full)
        # os.mkdir(Config.RAW_DIR)
        # os.mkdir(Config.TRAIN_DIR)
        # os.mkdir(Config.VAL_DIR)
        # os.mkdir(Config.EVAL_DIR)
        download_data_with_no_check(path_full, soup)

# calli_dir = args.calli_path
# calli_root = pathlib.Path(calli_dir)
# print("calli_root:", calli_root)

cids = args.urls_cid
format = args.url_format
all_calli_urls = [(str(cid), format + '&cid=' + str(cid)) for cid in cids]
total_num = len(all_calli_urls)
print(total_num)

for idx, (label, (cid, url)) in enumerate(zip(range(len(all_image_paths), len(all_image_paths) + len(all_calli_urls)), all_calli_urls)):
    print("Download \'{}\'... {} / {} ".format(cid, idx, total_num), url)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    folder_name = 'id_%d'%(label)

    path_full = os.path.join(args.save_path, folder_name)
    if not os.path.exists(path_full):
        os.mkdir(path_full)

    download_data(path_full, soup)

    # src_font = ImageFont.truetype(item, size=args.chara_size)
    # font_name = item.split('/')[-1].split('.')[0]
    # chars = get_char_list_from_ttf(item)  #
    # img_cnt = 0
    # for (chara, cnt) in zip(characters, range(len(characters))):
    #     img = draw_example(chara, src_font, args.img_size, (args.img_size-args.chara_size)/2, (args.img_size-args.chara_size)/2)
    #     path_full = os.path.join(args.save_path, 'id_%d'%(label))
    #     if not os.path.exists(path_full):
    #         os.mkdir(path_full)
    #     if np.sum(np.array(img) / 255.) < 18000:
    #         img_cnt += 1
    #         img.save(os.path.join(path_full, "%05d.png" % (cnt)))

10
Download '38'... 0 / 10  http://163.20.160.14/~word/modules/myalbum/viewcat.php?num=300&cid=38
data dir not empty, skipping download
Download '27'... 1 / 10  http://163.20.160.14/~word/modules/myalbum/viewcat.php?num=300&cid=27
data dir not empty, skipping download
Download '40'... 2 / 10  http://163.20.160.14/~word/modules/myalbum/viewcat.php?num=300&cid=40
data dir not empty, skipping download
Download '37'... 3 / 10  http://163.20.160.14/~word/modules/myalbum/viewcat.php?num=300&cid=37
data dir not empty, skipping download
Download '29'... 4 / 10  http://163.20.160.14/~word/modules/myalbum/viewcat.php?num=300&cid=29
data dir not empty, skipping download
Download '27'... 5 / 10  http://163.20.160.14/~word/modules/myalbum/viewcat.php?num=300&cid=27
data dir not empty, skipping download
Download '34'... 6 / 10  http://163.20.160.14/~word/modules/myalbum/viewcat.php?num=300&cid=34
data dir not empty, skipping download
Download '39'... 7 / 10  http://163.20.160.14/~word/modules/myalbu

In [6]:
# import argparse
from utils import dist_util, logger
from utils.image_datasets import load_data
from utils.resample import create_named_schedule_sampler
from utils.script_util import (
    model_and_diffusion_defaults,
    args_to_dict,
    create_model_and_diffusion,
)
from utils.train_util import TrainLoop
import torch as th
from attrdict import AttrDict
import yaml

ModuleNotFoundError: No module named 'blobfile'

In [ ]:


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--cfg_path', type=str, default='./cfg/train_cfg.yaml',
                        help='config file path')
    parser = parser.parse_args()
    with open(parser.cfg_path, 'r') as f:
        cfg = yaml.load(f, Loader=yaml.FullLoader)
    cfg = AttrDict(create_cfg(cfg))
    train_step = cfg.train_step
    total_train_step = cfg.total_train_step
    sty_encoder_path = cfg.sty_encoder_path
    classifier_free = cfg.classifier_free
    cfg.__delattr__('train_step')
    cfg.__delattr__('total_train_step')
    cfg.__delattr__('sty_encoder_path')
    cfg.__delattr__('classifier_free')

    dist_util.setup_dist()

    model_save_dir = cfg.model_save_dir  

    if not os.path.exists(model_save_dir):
        os.mkdir(model_save_dir)

    logger.configure(dir=model_save_dir, format_strs=['stdout', 'log', 'csv']) 

    logger.log("creating model and diffusion...")
    model, diffusion = create_model_and_diffusion(
        **args_to_dict(cfg, model_and_diffusion_defaults().keys())
    )
    model.to(dist_util.dev())
    schedule_sampler = create_named_schedule_sampler(cfg.schedule_sampler, diffusion)

    ### sty_encoder
    if not cfg.resume_checkpoint:
        logger.log("loading pre-trained model...")
        checkpoint = th.load(sty_encoder_path, map_location='cpu')
        tmp_dict = {}
        for k, v in checkpoint.items():
            if k in model.sty_encoder.state_dict():
                tmp_dict[k] = v
        model.sty_encoder.load_state_dict(tmp_dict)

        if classifier_free:
            raise ValueError(f"required conditional trained model, please fill in the model path in 'resume_checkpoint'")

    # frozen sty_encoder
    for p in model.sty_encoder.parameters():
        p.requires_grad = False


    logger.log("creating data loader...")
    data = load_data(
        data_dir=cfg.data_dir,
        batch_size=cfg.batch_size,
        image_size=cfg.image_size,
        stroke_path=cfg.stroke_path,
        classifier_free=classifier_free,
    )

    logger.log("training...")
    TrainLoop(
        model=model,
        diffusion=diffusion,
        data=data,
        batch_size=cfg.batch_size,
        microbatch=cfg.microbatch,
        lr=cfg.lr,
        ema_rate=cfg.ema_rate,
        log_interval=cfg.log_interval,
        save_interval=cfg.save_interval,
        train_step=train_step,
        resume_checkpoint=cfg.resume_checkpoint,
        use_fp16=cfg.use_fp16,
        fp16_scale_growth=cfg.fp16_scale_growth,
        schedule_sampler=schedule_sampler,
        weight_decay=cfg.weight_decay,
        classifier_free=classifier_free,
        total_train_step=total_train_step
    ).run_loop()


def create_cfg(cfg):
    defaults = dict(
        data_dir="",
        schedule_sampler="uniform",
        lr=1e-4,
        weight_decay=0.0,
        lr_anneal_steps=0,
        batch_size=1,
        microbatch=-1,
        ema_rate="0.9999",
        log_interval=250,
        save_interval=20000,
        resume_checkpoint="",
        use_fp16=False,
        fp16_scale_growth=1e-3,
        stroke_path=None,
        attention_resolutions='40, 20, 10',
    )
    defaults.update(model_and_diffusion_defaults())
    defaults.update(cfg)
    return defaults


if __name__ == "__main__":
    import os
    main()